In [7]:
from nltk.corpus import words
word_list = words.words?

This is an attempt to look at the differences between two words based off of the keyboard layout of your computer. It uses graph theory to look at the shortest path between two characters on your keyboard.

**key_graph** : A dictionary of keyboard characters, where the values are lists of the neighbors.

**breadth_first** : A simple breadth first search algorithm to find the shortest path between characters in key_graph.

**find_difference** : Breaks words/word segments up into single characters then passes them into the breadth first algorithm and then finds legth of the shortest path, that path is the character's difference 'score'. The scores are summed for the entire word and divided by the length of the word to give a word/segment difference score.

**compare_words** : Compares the differences between the lengths of the two words, if they are different, passes them into the `find_differences` function by shortest word chunk of the longer word. eg: `compare_words("cat", "course")` passes `("cat", "cou")` ,  `("cat", "our")`, `("cat", "urs")`, and `("cat", "rse")` into `find_differences`. Then finds the best scoring segment, then adds a penalty of $\frac{|len(w1) - len(w2)|}{len(w1)}$ to the lowest score.

In [2]:
key_graph = {"q" : ["w", "a"],
             "w" : ["q", "a", "s", "e"],
             "e" : ["w", "s", "d", "r"],
             "r" : ["e", "d", "f", "t"],
             "t" : ["r", "f", "g", "y"],
             "y" : ["t", "g", "h", "u"],
             "u" : ["y", "h", "j", "i"],
             "i" : ["u", "j", "k", "o"],
             "o" : ["i", "k", "l", "p"],
             "p" : ["o", "l"],
             "a" : ["q", "w", "s", "z"],
             "s" : ["a", "w", "e", "d", "x", "z"],
             "d" : ["s", "e", "r", "f", "c", "x"],
             "f" : ["d", "r", "t", "g", "v", "c"],
             "g" : ["f", "t", "y", "h", "b", "v"],
             "h" : ["g", "y", "u", "j", "n", "b"],
             "j" : ["h", "u", "i", "k", "m", "n"],
             "k" : ["j", "i", "o", "l", "m"],
             "l" : ["k", "o", "p"],
             "z" : ["a", "s", "x"],
             "x" : ["z", "s", "d", "c"],
             "c" : ["x", "d", "f", "v"],
             "v" : ["c", "f", "g", "b"],
             "b" : ["v", "g", "h", "n"],
             "n" : ["b", "h", "j", "m"],
             "m" : ["n", "j", "k"]}

In [27]:
def breadth_first(graph,start, end):
    """
    Breadth First Search Algorithm.
    Returns shortest path.
    -------------------------------------
    Inputs:
        graph : Dictionary of connecting nodes
        start : Starting point on the graph
        end   : Ending point on the graph
    Outputs:
        path  : List of shortest path
    """
    queue = []
    queue.append([start])
    while queue:
        path = queue.pop(0)
        node = path[-1]
        if node == end:
            return path
        for adjacent in graph.get(node, []):
            new_path = list(path)
            new_path.append(adjacent)
            queue.append(new_path)

In [39]:
def find_difference(seg1, seg2):
    letter_score = []
    for c1,c2 in zip(seg1, seg2):
        letter_score.append(float(len(breadth_first(key_graph, c1, c2)) - 1))
    return sum(letter_score)/len(letter_score)

In [40]:
def compare_words(word1, word2):
    word1 = word1.lower()
    word2 = word2.lower()
    seg_scores = []
    if len(word1) >= len(word2):
        for i in range(0, len(word1) - len(word2) + 1):
            seg_scores.append(find_difference(word1[i:i+len(word2)], word2))
    else:
        for i in range(0, len(word2) - len(word1) + 1):
            seg_scores.append(find_difference(word1[i:i+len(word2)], word2))
    return round(min(seg_scores) + abs(len(word1) - len(word2))/float(len(word1)),2)

# Example

In [51]:
from pymongo import MongoClient
import pprint as pp
from nltk.corpus import stopwords
import nltk
import numpy as np
client = MongoClient()
db = client.nyt_dump
coll = db.articles

In [52]:
stopwords_ = set(stopwords.words('english'))

In [53]:
keywords = {}
for num, document in enumerate(coll.find()):
    temp_content = ''.join(document['content']).encode('ascii','ignore').replace('\n', '')
    tokens = nltk.word_tokenize(temp_content)
    result = [x for x in tokens if x not in stopwords_ and x.isalpha()]
    keywords[num] = result

In [61]:
word_list = set()
for l in keywords.values():
    word_list = word_list | set(l)

In [63]:
word_list = list(word_list)

In [37]:
checkword = "covfefe"
thresh = 1 #How many letters am I willing to go out to.

In [79]:
best_words = []
for ind, word in enumerate(word_list):
    if ind % 100 == 0:
        print ind
    if len(word) <= (len(checkword) + thresh) and len(word) >= (len(checkword) - thresh):
        word_score = compare_words(checkword, word)
        if word_score <= 2.0:
            best_words.append([word_score,word])

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000


KeyboardInterrupt: 

In [80]:
len(best_words)

273

In [81]:
len(word_list)

39335

In [87]:
sorted(best_words)[:31]

[[0.64, 'coffee'],
 [0.71, 'conceded'],
 [0.71, 'differed'],
 [0.86, 'corseted'],
 [0.98, 'overseas'],
 [1.0, 'contend'],
 [1.0, 'corrects'],
 [1.0, 'diverse'],
 [1.14, 'Cutter'],
 [1.14, 'Preece'],
 [1.14, 'bothers'],
 [1.14, 'buffer'],
 [1.14, 'courses'],
 [1.14, 'ringers'],
 [1.14, 'sitters'],
 [1.29, 'Couture'],
 [1.29, 'budgets'],
 [1.29, 'contents'],
 [1.29, 'couture'],
 [1.29, 'hovered'],
 [1.31, 'Indeed'],
 [1.31, 'Odette'],
 [1.31, 'Orsett'],
 [1.31, 'Rouget'],
 [1.31, 'binder'],
 [1.31, 'funded'],
 [1.31, 'invert'],
 [1.31, 'onscreen'],
 [1.31, 'peters'],
 [1.31, 'singer'],
 [1.43, 'Concerns']]